In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision, MeanIoU
from model import deeplabv3_plus
# from metrics import dice_loss, dice_coef, iou
from cityscapes_dataloader import Cityscapes, get_img_paths

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


In [2]:
input_dir = "cityscapes\\leftImg8bit\\train"
target_dir = "cityscapes\\gtFine\\train"
images, annotations = get_img_paths(input_dir, target_dir)
# print(f"Images: {len(images)} - Annotations: {len(annotations)}")

val_input_dir = "cityscapes\\leftImg8bit\\val"
val_target_dir = "cityscapes\\gtFine\\val"
val_images, val_annotations = get_img_paths(val_input_dir, val_target_dir)
# print(f"val_Images: {len(val_images)} - val_Annotations: {len(val_annotations)}")

image_paths = images + val_images
annotation_paths = annotations + val_annotations
print(f"Images: {len(image_paths)} - Annotations: {len(annotation_paths)} {image_paths[0]} {annotation_paths[0]}")

test_input_dir = "cityscapes\\leftImg8bit\\test"
test_target_dir = "cityscapes\\gtFine\\test"
test_image_paths, test_annotation_paths = get_img_paths(test_input_dir, test_target_dir)
print(f"Images: {len(test_image_paths)} - Annotations: {len(test_annotation_paths)} {test_image_paths[0]} {test_annotation_paths[0]}")

img_size = (512, 512)
H, W = img_size
num_classes = 24
batch_size = 32
lr = 1e-4
num_epochs = 20

create_dir('files')
model_path = os.path.join('files', 'model.h5')
csv_path = os.path.join('files', 'data.csv')

Images: 3475 - Annotations: 3475 cityscapes\leftImg8bit\train\aachen\aachen_000000_000019_leftImg8bit.png cityscapes\gtFine\train\aachen\aachen_000000_000019_gtFine_labelIds.png
Images: 1525 - Annotations: 1525 cityscapes\leftImg8bit\test\berlin\berlin_000000_000019_leftImg8bit.png cityscapes\gtFine\test\berlin\berlin_000000_000019_gtFine_labelIds.png


In [3]:
train_gen = Cityscapes(batch_size=batch_size, img_size=img_size, input_img_paths=image_paths, target_img_paths=annotation_paths)
test_gen = Cityscapes(batch_size=batch_size, img_size=img_size, input_img_paths=test_image_paths, target_img_paths=test_annotation_paths)

In [4]:
model = deeplabv3_plus((H, W, 3))
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr), metrics=[MeanIoU(num_classes=24), Recall(), Precision()])
model.summary()

Shape of x_a: (None, 128, 128, 256)
Shape of x_b: (None, 128, 128, 48)
Shape of x: (None, 128, 128, 304)
Shape of x: (None, 128, 128, 256)
Shape of x: (None, 512, 512, 24)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 518, 518, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 256, 256, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormal

In [5]:
callbacks = [
  ModelCheckpoint(model_path, verbose=1, save_best_only=True),
  ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
  CSVLogger(csv_path),
  TensorBoard(),
  EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
]

In [6]:
model.fit(
  train_gen,
  epochs=num_epochs,
  validation_data=test_gen,
  callbacks=callbacks
)

C:\Users\kcy96\anaconda3\envs\cy\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 1/20


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  OOM when allocating tensor with shape[32,64,256,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model/conv1_conv/Conv2D (defined at <ipython-input-6-dca0328ec6de>:1) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

	 [[confusion_matrix/assert_less_1/Assert/AssertGuard/pivot_f/_31/_123]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

  (1) Resource exhausted:  OOM when allocating tensor with shape[32,64,256,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model/conv1_conv/Conv2D (defined at <ipython-input-